In [107]:
from openpyxl import Workbook
from openpyxl import load_workbook
from bs4 import BeautifulSoup
from selenium import webdriver

In [108]:
dict_field_num = {'잠실':'1', '고척':'2', '수원':'3', '사직':'4', '광주':'5', '문학':'6', '창원':'7', '마산':'7', '대구':'8', '대전':'9', '울산':'10', '포항':'11', '청주':'12'}
dict_team_num = {'KT':'1', 'LG':'2', 'SK':'3', 'NC':'4', 'KIA':'5', '삼성':'6', '롯데':'7', '한화':'8', '두산':'9', '넥센':'10', '키움':'10', '드림':'11', '나눔':'12'}

In [109]:
# 선수이름&선수생년월일  데이터를 통해 선수의 woba가져오기
def getBatterWoba(player, dict_player):
    if dict_player.get(player) != None:
        print('있음 : ' + player)
        return dict_player[player]
    
    url = 'http://www.statiz.co.kr/player.php?name='+player.split('&')[0]+'&birth='+player.split('&')[1]
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')

    #woba가 있는 <td>태그에 접근
    woba = soup.select_one('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(1) > div > div.box-body.no-padding.table-responsive > table > tbody > tr.evenrow_stz0 > td:nth-child(29)')

    #woba가 제대로 나와있지 않은 데이터에 대한 처리
    if woba == None:
        print(player + ':none')
        woba = '0.000'
    elif woba.text =='':
        print(player + ':')
        woba = '0.000'
    elif woba.text=='0.000':
        print(player +':0.000')
        woba = '0.000'
    else:
        woba = woba.text
        
    #woba값 dict_player라는 딕셔너리 변수에 저장
    dict_player[player] = woba
    driver.quit()
    return woba
    

In [110]:
#경기 일정 및 경기 결과에 대한 정보와 선수들의 타순과 날씨를 파일에서 읽어서 수치화한다.
#getBatter()라는 함수로 기록된 18개의 타자마다 woba를 구한다.
#이미 woba를 구했으면 웹크롤링에서 제외하기 위해서 dict_player에 기록해둔다.
def numerizeMergeFile(filename, dict_match_numerized, dict_player):
    if type(dict_match_numerized) != type({}):
        print('put dict_match_numerized as a dictionary; {\'key\':\'value\'}')
        return
    book = load_workbook(filename)
    sheet = book.active
    rows = list(sheet.rows)[1:]
    #key : year -> month -> day -> time -> field = []  ; <- 딕셔너리 구조
    index = 0
    # 라벨링(승리, 무승부, 패)
    label_win = 0
    label_draw = 0.5
    label_lose = 1
    # 엑셀 파일의 각 행마다 수치화진행 (즉 각 샘플마다)
    for row in rows:
        #ex) ymd = 1995-04-25
        ymd = row[0].value
        ymd = int(ymd.split('-')[0] + ymd.split('-')[1] + ymd.split('-')[2])

        #field : 구장에 해당하는 번호
        field = int(dict_field_num[row[1].value])

        #해당 경기가 치뤄진 시작 시간
        time = row[2].value
        #print(str(ymd) +', ' + str(field) + ', ' + time)
        time = float(time[0:2]) + float(time[3:])/60

        #이긴팀, 진팀에 대한 정보 임시 저장 (팀스코어 제외)
        #[0] : 팀인덱스, [1] : 팀 스코어, [2] : 홈어웨이 여부
        list_winteam = []
        list_winteam.append(int(dict_team_num[row[3].value]))
        #list_winteam.append(int(row[4].value))
        list_winteam.append(int(row[5].value))

        list_loseteam = []
        list_loseteam.append(int(dict_team_num[row[6].value]))
        #list_loseteam.append(int(row[7].value))
        list_loseteam.append(int(row[8].value))
        
        # 타자의 woba를 가져오는 코드 (임시 저장)
        list_player_woba = []
        for player in row[9:26]:
            print(player.value)
            list_player_woba.append(getBatterWoba(player.value, dict_player))
        
        
        
        #[0] : 평균기온(섭씨), [1] : 강수량(mm), [2] : 평균풍속(km/h), [3] : 상대습도(%)
        # 고척 : '-', '-', '-', '-'  =>  20, 0.0, 0.0, 60 으로 임의 설정.
        list_weather = []
        if row[27].value != '-':
            for idx in range(4):
                list_weather.append(float(row[idx+27].value))
        else:
            list_weather.append(20)
            list_weather.append(0.0)
            list_weather.append(0.0)
            list_weather.append(60)

            
        #index 이기는 팀이 왼쪽, labeling 1 \ index+1 지는 팀이 왼쪽, labeling 0
        dict_match_numerized[index] = []
        dict_match_numerized[index+1] = []
        #라벨링
        if list_winteam[1] == list_loseteam[1]:
            dict_match_numerized[index].append(label_draw)
            dict_match_numerized[index+1].append(label_draw)
        else:
            dict_match_numerized[index].append(label_win)
            dict_match_numerized[index+1].append(label_lose)

        #연도, 구장, 시간 기록 (index는 이긴팀, index+1은 진팀)
        dict_match_numerized[index].append(ymd)
        dict_match_numerized[index].append(field)
        dict_match_numerized[index].append(time)
        dict_match_numerized[index+1].append(ymd)
        dict_match_numerized[index+1].append(field)
        dict_match_numerized[index+1].append(time)

        #이긴팀 진팀에 대한 정보기록
        for elem in list_winteam:
            dict_match_numerized[index].append(elem)
        for elem in list_loseteam:
            dict_match_numerized[index].append(elem)
            dict_match_numerized[index+1].append(elem)
        for elem in list_winteam:
            dict_match_numerized[index+1].append(elem)
        
        #woba기록
        for elem in list_player_woba:
            dict_match_numerized[index].append(elem)
            dict_match_numerized[index+1].append(elem)
        #날씨 기록    
        for elem in list_weather:
            dict_match_numerized[index].append(elem)
            dict_match_numerized[index+1].append(elem)

        index += 2

In [111]:
#수치화된 데이터 저장 
def writeFileFromNumerizedData(filename, dict_match_numerized):
    #임시 columns (attributes)
    #index(order) label 년월일 시각 팀번호 팀점수 팀홈어웨이 상대팀번호 상대팀점수 상대팀홈어웨이 날씨[평균기온 강수량 평균풍속 상대습도] 
    book = Workbook()
    sheet = book.active
    sheet.title = filename.split('.')[0]
    #sheet.append(['index', 'label', 'ymd', 'field', 'time', '팀번호', '팀점수', '팀홈어웨이', '상대팀번호', '상대팀점수', '상대팀홈어웨이', '평균기온', '강수량', '평균풍속', '상대습도'])
    sheet.append(['label', 'ymd', 'field', 'time', '팀번호', '팀홈어웨이', '상대팀번호', '상대팀홈어웨이', 'woba1', 'woba1','woba1','woba1','woba1','woba1','woba1','woba1','woba1','woba0', '평균기온', '강수량', '평균풍속', '상대습도'])
    for index in dict_match_numerized.keys():
        list_temp = dict_match_numerized[index]
#        list_temp.insert(0, index)
        sheet.append(list_temp)
    book.save(filename)

In [112]:
filename = 'merge2.xlsx'
dict_match_numerized = {}
dict_player = {}
#수치화하는 함수 부르기
numerizeMergeFile(filename, dict_match_numerized, dict_player)

허경민&1990-08-26
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><span style="padding-left:3px;padding-right:3px;">0.373</span></td>
정수빈&1990-10-07
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><span style="padding-left:3px;padding-right:3px;">0.342</span></td>
박건우&1990-09-08
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><span style="padding-left:3px;padding-right:3px;">0.377</span></td>
김재환&1988-09-22
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><span style="padding-left:3px;padding-right:3px;">0.389</span></td>
오재일&1986-10-29
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><span style="padding-left:3px;padding-right:3px;">0.391</span></td>
페르난데스&1988-04-27
<td class="statdata" style="text-align

<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.345</span></font></td>
김혜성&1999-01-27
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><span style="padding-left:3px;padding-right:3px;">0.345</span></td>
주효상&1997-11-11
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.257</span></font></td>
민병헌&1987-03-10
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.271</span></font></td>
손아섭&1988-03-18
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><span style="padding-left:3px;padding-right:3px;">0

<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.371</span></font></td>
이명기&1987-12-26
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><span style="padding-left:3px;padding-right:3px;">0.354</span></td>
나지완&1985-05-19
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><span style="padding-left:3px;padding-right:3px;">0.378</span></td>
한승택&1994-06-21
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.301</span></font></td>
정근우&1982-10-02
있음 : 정근우&1982-10-02
장진혁&1993-09-30
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-le

<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.292</span></font></td>
최원준&1997-03-23
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.359</span></font></td>
정근우&1982-10-02
있음 : 정근우&1982-10-02
양성우&1989-05-02
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.209</span></font></td>
송광민&1983-06-24
있음 : 송광민&1983-06-24
호잉&1989-05-18
있음 : 호잉&1989-05-18
김태균&1982-05-29
있음 : 김태균&1982-05-29
이성열&1984-07-13
있음 : 이성열&1984-07-13
정은원&2000-01-17
있음 : 정은원&2000-01-17
하주석&1994-02-25
있음 : 하주석&1994-02-25
허경민&1990-08-26
있음 : 허경민&1990-08-26
페르난데스&1988-04-27
있음 : 페르난데스&1988-04-27
박건우&1990-09-08
있음 : 박건우&1990-09-08
김재환&1988-

<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.315</span></font></td>
양종민&1990-10-09
있음 : 양종민&1990-10-09
오지환&1990-03-12
있음 : 오지환&1990-03-12
민병헌&1987-03-10
있음 : 민병헌&1987-03-10
손아섭&1988-03-18
있음 : 손아섭&1988-03-18
전준우&1986-02-25
있음 : 전준우&1986-02-25
이대호&1982-06-21
있음 : 이대호&1982-06-21
채태인&1982-10-11
있음 : 채태인&1982-10-11
한동희&1999-06-01
있음 : 한동희&1999-06-01
아수아헤&1991-11-02
있음 : 아수아헤&1991-11-02
김준태&1994-07-31
있음 : 김준태&1994-07-31
허경민&1990-08-26
있음 : 허경민&1990-08-26
페르난데스&1988-04-27
있음 : 페르난데스&1988-04-27
박건우&1990-09-08
있음 : 박건우&1990-09-08
김재환&1988-09-22
있음 : 김재환&1988-09-22
오재일&1986-10-29
있음 : 오재일&1986-10-29
오재원&1985-02-09
있음 : 오재원&1985-02-09
김재호&1985-03-21
있음 : 김재호&1985-03-21
박세혁&1990-01-09
있음 : 박세혁&1990-01-09
정수빈&1990-10-07
있음 : 정수빈&1990-10-07
박해민&1990-02-24
있음 : 박해민&1990-02-24
박한이&1979-01-28
있음 : 박한이&1979-01-28
구자욱&1993-02-12
있음 : 구자욱&1993-02-12
러프&1986-07-28
있음 : 러프

...

<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.274</span></font></td>
노수광&1990-08-06
있음 : 노수광&1990-08-06
고종욱&1989-01-11
있음 : 고종욱&1989-01-11
최정&1987-02-28
있음 : 최정&1987-02-28
한동민&1989-08-09
있음 : 한동민&1989-08-09
정의윤&1986-07-25
있음 : 정의윤&1986-07-25
로맥&1985-09-30
있음 : 로맥&1985-09-30
이재원&1988-02-24
있음 : 이재원&1988-02-24
김창평&2000-06-14
있음 : 김창평&2000-06-14
김성현&1987-03-09
있음 : 김성현&1987-03-09
이천웅&1988-10-20
있음 : 이천웅&1988-10-20
오지환&1990-03-12
있음 : 오지환&1990-03-12
이형종&1989-06-07
있음 : 이형종&1989-06-07
김현수&1988-01-12
있음 : 김현수&1988-01-12
채은성&1990-02-06
있음 : 채은성&1990-02-06
페게로&1987-02-22
있음 : 페게로&1987-02-22
김민성&1988-12-17
있음 : 김민성&1988-12-17
유강남&1992-07-15
있음 : 유강남&1992-07-15
김태진&1995-10-07
있음 : 김태진&1995-10-07
이명기&1987-12-26
있음 : 이명기&1987-12-26
박민우&1993-02-06
있음 : 박민우&1993-02-06
양의지&1987-06-05
있음 : 양의지&1987-06-05
박석민&1985-06-22
있음 : 박석민&1985-06-22
모창민&1985-05-08
있음 : 모창민&1985-05

<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.182</span></font></td>
김동한&1988-06-24
있음 : 김동한&1988-06-24
정은원&2000-01-17
있음 : 정은원&2000-01-17
장진혁&1993-09-30
있음 : 장진혁&1993-09-30
송광민&1983-06-24
있음 : 송광민&1983-06-24
이성열&1984-07-13
있음 : 이성열&1984-07-13
김회성&1985-12-04
있음 : 김회성&1985-12-04
최재훈&1989-08-27
있음 : 최재훈&1989-08-27
김민하&1989-02-25
있음 : 김민하&1989-02-25
최진행&1985-08-17
있음 : 최진행&1985-08-17
손아섭&1988-03-18
있음 : 손아섭&1988-03-18
정훈&1987-07-18
있음 : 정훈&1987-07-18
민병헌&1987-03-10
있음 : 민병헌&1987-03-10
전준우&1986-02-25
있음 : 전준우&1986-02-25
윌슨&1990-07-29
있음 : 윌슨&1990-07-29
신본기&1989-03-21
있음 : 신본기&1989-03-21
한동희&1999-06-01
있음 : 한동희&1999-06-01
정보근&1999-08-31
있음 : 정보근&1999-08-31
강로한&1992-05-13
있음 : 강로한&1992-05-13
박찬호&1995-06-05
있음 : 박찬호&1995-06-05
김선빈&1989-12-18
있음 : 김선빈&1989-12-18
터커&1990-07-06
있음 : 터커&1990-07-06
최형우&1983-12-16
있음 : 최형우&1983-12-16
유민상&1989-04-13
있음 : 유민상&1989-04-1

<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.227</span></font></td>
조용호&1989-09-09
있음 : 조용호&1989-09-09
오승택&1991-11-18
있음 : 오승택&1991-11-18
로하스&1990-05-24
있음 : 로하스&1990-05-24
강백호&1999-07-29
있음 : 강백호&1999-07-29
박경수&1984-03-31
있음 : 박경수&1984-03-31
황재균&1987-07-28
있음 : 황재균&1987-07-28
장성우&1990-01-17
있음 : 장성우&1990-01-17
배정대&1995-06-12
있음 : 배정대&1995-06-12
심우준&1995-04-28
있음 : 심우준&1995-04-28
서건창&1989-08-22
있음 : 서건창&1989-08-22
이정후&1998-08-20
있음 : 이정후&1998-08-20
샌즈&1987-09-28
있음 : 샌즈&1987-09-28
박병호&1986-07-10
있음 : 박병호&1986-07-10
김하성&1995-10-17
있음 : 김하성&1995-10-17
김규민&1993-01-26
있음 : 김규민&1993-01-26
김혜성&1999-01-27
있음 : 김혜성&1999-01-27
임병욱&1995-09-30
있음 : 임병욱&1995-09-30
정은원&2000-01-17
있음 : 정은원&2000-01-17
장진혁&1993-09-30
있음 : 장진혁&1993-09-30
김태균&1982-05-29
있음 : 김태균&1982-05-29
이성열&1984-07-13
있음 : 이성열&1984-07-13
정근우&1982-10-02
있음 : 정근우&1982-10-02
최진행&1985-08-17
있음 : 최진행&1985-

<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.000</span></font></td>
이정훈&1994-12-07:0.000
고장혁&1990-01-08
있음 : 고장혁&1990-01-08
박해민&1990-02-24
있음 : 박해민&1990-02-24
윌리엄슨&1990-07-15
있음 : 윌리엄슨&1990-07-15
구자욱&1993-02-12
있음 : 구자욱&1993-02-12
러프&1986-07-28
있음 : 러프&1986-07-28
이원석&1986-10-21
있음 : 이원석&1986-10-21
이학주&1990-11-04
있음 : 이학주&1990-11-04
김헌곤&1988-11-09
있음 : 김헌곤&1988-11-09
박계범&1996-01-11
있음 : 박계범&1996-01-11
김준완&1991-01-20
<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.292</span></font></td>
이상호&1989-02-05
있음 : 이상호&1989-02-05
박민우&1993-02-06
있음 : 박민우&1993-02-06
양의지&1987-06-05
있음 : 양의지&1987-06-05
스몰린스키&1989-02-09
있음 : 스몰린스키&1989-02-09
박석민&1985-06-22
있음 : 박석민&1985-06-22
노진혁&1989-07-15
있음 : 노진혁&1989-07-15
권희동&1990-12-30
있음 : 권희동&1990-

<td class="statdata" style="text-align:right;white-space:nowrap;text-align:center;vertical-align:middle;"><font color="#4B4B4B"><span style="padding-left:3px;padding-right:3px;">0.258</span></font></td>
전준우&1986-02-25
있음 : 전준우&1986-02-25
이대호&1982-06-21
있음 : 이대호&1982-06-21
윌슨&1990-07-29
있음 : 윌슨&1990-07-29
신본기&1989-03-21
있음 : 신본기&1989-03-21
김민수&1998-03-18
있음 : 김민수&1998-03-18
정보근&1999-08-31
있음 : 정보근&1999-08-31
허경민&1990-08-26
있음 : 허경민&1990-08-26
페르난데스&1988-04-27
있음 : 페르난데스&1988-04-27
박건우&1990-09-08
있음 : 박건우&1990-09-08
오재일&1986-10-29
있음 : 오재일&1986-10-29
김재환&1988-09-22
있음 : 김재환&1988-09-22
박세혁&1990-01-09
있음 : 박세혁&1990-01-09
김재호&1985-03-21
있음 : 김재호&1985-03-21
정수빈&1990-10-07
있음 : 정수빈&1990-10-07
류지혁&1994-01-13
있음 : 류지혁&1994-01-13
박해민&1990-02-24
있음 : 박해민&1990-02-24
윌리엄슨&1990-07-15
있음 : 윌리엄슨&1990-07-15
구자욱&1993-02-12
있음 : 구자욱&1993-02-12
러프&1986-07-28
있음 : 러프&1986-07-28
이원석&1986-10-21
있음 : 이원석&1986-10-21
이성규&1993-08-03
있음 : 이성규&1993-08-03
최영진&1988-05-10
있음 : 최영진&1988-05-10
손주인&1983-12-01
있음 : 손주인&1

In [113]:
#filename으로 수치화된 정보 파일에 쓰기
filename = 'numerized_merge2.xlsx'
writeFileFromNumerizedData(filename, dict_match_numerized)